In [1]:
import pandas as pd
import numpy as np
import optuna

In [2]:
train = pd.read_csv("data/train.csv")
train.head()

,id,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,0,L50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0,0
1,1,M20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0,0
2,2,L49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0,0
3,3,L53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0,0
4,4,M24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0,0


In [3]:
cols = ['Product ID','Type','Air temperature [K]','Process temperature [K]','Rotational speed [rpm]','Torque [Nm]','Tool wear [min]','TWF',
       'HDF','PWF','OSF','RNF']
X = train[cols]
y = train['Machine failure']

In [4]:
X.isnull().sum()

Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

In [5]:
X.shape

(136429, 12)

In [6]:
X['Product ID'] = X['Product ID'].str[1:6]
X.head()

C:\Users\lijin\AppData\Local\Temp\ipykernel_9648\244534733.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Product ID'] = X['Product ID'].str[1:6]


,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],TWF,HDF,PWF,OSF,RNF
0,50096,L,300.6,309.6,1596,36.1,140,0,0,0,0,0
1,20343,M,302.6,312.1,1759,29.1,200,0,0,0,0,0
2,49454,L,299.3,308.5,1805,26.5,25,0,0,0,0,0
3,53355,L,301.0,310.9,1524,44.3,197,0,0,0,0,0
4,24050,M,298.0,309.0,1641,35.4,34,0,0,0,0,0


In [7]:
y.value_counts()

0    134281
1      2148
Name: Machine failure, dtype: int64

In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import VotingRegressor
from sklearn.compose import make_column_selector,make_column_transformer
import lightgbm as lgb
from catboost import CatBoostClassifier
import xgboost as xgb
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import balanced_accuracy_score,log_loss
from sklearn.impute import KNNImputer
from sklearn.model_selection import cross_val_score

In [7]:
# def objective(trial):
#     lgbm_params = {
#         'boosting_type': trial.suggest_categorical('lgbm_boosting_type', ['goss', 'dart','gbdt']),
#         'learning_rate': trial.suggest_float('lgbm_learning_rate', 0.01, 0.2),
#         'n_estimators': trial.suggest_int('lgbm_n_estimators', 100, 1000),
#         'num_leaves': trial.suggest_int('lgbm_num_leaves', 16, 64),
#         'random_state': 3097,
#         'min_child_samples': trial.suggest_int('lgbm_min_child_samples', 1, 20),
#         'reg_alpha': trial.suggest_float('lgbm_reg_alpha', 0.0, 1.0),
#         'reg_lambda': trial.suggest_float('lgbm_reg_lambda', 0.0, 1.0)
#     }

#     num_cols = make_column_selector(dtype_include='number')
#     cat_cols = make_column_selector(dtype_exclude='number')

#     preprocessor = make_column_transformer(
#         (StandardScaler(), num_cols),
#         (OneHotEncoder(handle_unknown='ignore'), cat_cols)
#     )

#     lgbm = lgb.LGBMClassifier(**lgbm_params, class_weight='balanced')

#     pipe = make_pipeline(preprocessor, lgbm)
    
#     acc = cross_val_score(pipe,X,y,scoring='balanced_accuracy').mean()
#     return acc

In [ ]:
# #创建Optuna优化器和目标函数
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=5)

# # 输出最佳参数和得分
# best_params = study.best_params
# best_score = study.best_value
# print('Best Parameters:', best_params)
# print('Best Balanced Accuracy:', best_score) #0.30

- Tune 2

- Params of XGBoost
    - 'n_estimators': 1500,
    - 'objective': 'binary:logistic', 
    - 'eval_metric': 'auc', 
    - 'random_state': 42, 
    - 'learning_rate' : 0.09719877907815099, 
    - 'subsample' : 0.9225106653522045, 
    - 'colsample_bytree' : 0.212545425027345,
    - 'max_depth': 8,
    - 'early_stopping_rounds': 100

- Params of Catboost

    'loss_function': 'Logloss',
         'eval_metric': 'AUC',
         'random_seed': 19970507,
         'learning_rate': 0.027, 
          'iterations': 927, 
          'depth': 5, 
          'subsample': 0.705

- Catboost Params

'loss_function': 'Logloss',
         'eval_metric': 'AUC',
         'random_seed': 19970507,
         'learning_rate': 0.027, 
          'iterations': 927, 
          'depth': 5, 
          'subsample': 0.705

In [9]:
from sklearn.model_selection import StratifiedKFold

In [10]:
def objective(trial):
    lgbm_params = {
        'boosting_type': trial.suggest_categorical('lgbm_boosting_type', ['goss', 'dart','gbdt']),
        'learning_rate': trial.suggest_float('lgbm_learning_rate', 0.05, 0.13),
        'n_estimators': trial.suggest_int('lgbm_n_estimators', 100, 1000),
        'num_leaves': trial.suggest_int('lgbm_num_leaves', 16, 64),
        'random_state': 318
#         'min_child_samples': trial.suggest_int('lgbm_min_child_samples', 1, 20),
#         'reg_alpha': trial.suggest_float('lgbm_reg_alpha', 0.0, 1.0),
#         'reg_lambda': trial.suggest_float('lgbm_reg_lambda', 0.0, 1.0)
    }
    
#     xgbc_params = {
#         'n_estimators': trial.suggest_int('xgbc_n_estimators', 100, 1000),
#         'random_state': 318, 
#         'learning_rate' : trial.suggest_float('xgbc_learning_rate', 0.05, 0.13), 
#         'subsample' : trial.suggest_float('xgbc_subsample', 0.9, 1), 
#         'colsample_bytree' : trial.suggest_float('xgbc_colsample_bytree', 0.2, 0.4),
#         'max_depth': trial.suggest_int('xgbc_max_depth', 3,10)
#     }
    
#     cbc_params = {
#         'eval_metric': 'AUC',
#         'random_seed': 318,
#         'learning_rate': trial.suggest_float('cbc_learning_rate', 0.02, 0.12), 
#         'iterations': trial.suggest_int('cbc_iterations', 500,1000), 
#         'depth': trial.suggest_int('cbc_depth', 3,10), 
#         'subsample': trial.suggest_float('cbc_subsample', 0.6, 0.8)
#     }
    
#     hgbc_params = {
#         'random_state': 318
#     }

    num_cols = make_column_selector(dtype_include='number')
    cat_cols = make_column_selector(dtype_exclude='number')

#     weight_lgbm = trial.suggest_int('weight_lgbm', 1, 2)
#     weight_xgbc = trial.suggest_int('weight_xgbc', 1, 2)
#     weight_cbr = trial.suggest_int('weight_cbr', 1, 2)
#     weight_hgbc = trial.suggest_int('weight_hgbc', 1, 2)

    preprocessor = make_column_transformer(
        (StandardScaler(), num_cols),
        (OneHotEncoder(handle_unknown='ignore'), cat_cols)
    )

    lgbm = lgb.LGBMClassifier(**lgbm_params, class_weight='balanced') #0.96440897
#     xgbc = xgb.XGBClassifier(scale_pos_weight=4.71,**xgbc_params,objective='binary:logistic',eval_metric='auc') #0.96625893
#     hgbc = HistGradientBoostingClassifier(**hgbc_params)
#     cbc = CatBoostClassifier(verbose=0, **cbc_params, auto_class_weights='Balanced') #0.96297647
    
#     clf = VotingClassifier(
#         estimators=[('lgbm', lgbm),  ('cbc', cbc), ('xgbc',xgbc),('hgbc',hgbc)],
#         #estimators=[('lgbm', lgbm), ('xgbc', xgbc)],
#         voting='soft',
#         weights=[1,1,1,1]
#     )
    
    pipe = make_pipeline(preprocessor, lgbm)
    
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=318)

    accs = []  # 用于存储每个折叠的损失值

    for train_index, val_index in skf.split(X, y):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        pipe.fit(X_train, y_train)

        y_pred = pipe.predict_proba(X_val)
        acc = cross_val_score(pipe,X,y,scoring='roc_auc').mean()
        accs.append(acc)
        
    avg_acc = np.mean(accs)

    return avg_acc

In [ ]:
#创建Optuna优化器和目标函数
study = optuna.create_study(direction='maximize')
%time study.optimize(objective, n_trials=10)

# 输出最佳参数和得分
best_params = study.best_params
best_score = study.best_value
print('Best Parameters:', best_params)
print('Best Balanced Accuracy:', best_score) #0.96454

[I 2023-06-23 23:07:26,609] A new study created in memory with name: no-name-5760a77e-7939-4f4a-80b1-8a45a76d7181
